In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 1. Get the data ready
heart_disease = pd.read_csv('../datos/heart-disease.csv')
# Create X (features matrix)
X = heart_disease.drop("target", axis=1)
# Create y (the target column)
y = heart_disease["target"]

In [3]:
y.value_counts()

1    165
0    138
Name: target, dtype: int64

In [35]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # test size=0.25 = 25%

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

## Model and Hyperparameters

In [5]:
# We'll use a Random Forest
from sklearn.ensemble import RandomForestClassifier
# as we are using a classifier model, we abbreviate as clf
clf = RandomForestClassifier()

In [6]:
# We'll leave the hyperparameters as default to begin with...
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# Manejo de warnings
# import warnings
# warnings.filterwarnings("default") # other options: "ignore"

In [7]:
# Training
clf.fit(X_train, y_train)

RandomForestClassifier()

## Predicition

In [8]:
# Use the model to make a prediction on the test data (further evaluation)
y_preds = clf.predict(X_test)
# show the predictions
y_preds

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

## Evaluation

In [9]:
# Evaluate the model on the training set
# should be a perfect score since we are evaluationg based one the training
clf.score(X_train, y_train)

1.0

In [10]:
# Evaluate the model on the test set
clf.score(X_test, y_test)

0.8026315789473685

In [11]:
# Apart from score() There are also a number of other evaluation methods we can use for our models.
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [12]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.80      0.78      0.79        36
           1       0.80      0.82      0.81        40

    accuracy                           0.80        76
   macro avg       0.80      0.80      0.80        76
weighted avg       0.80      0.80      0.80        76



In [13]:
print(confusion_matrix(y_test, y_preds))

[[28  8]
 [ 7 33]]


In [14]:
print(accuracy_score(y_test, y_preds))

0.8026315789473685


## Tunning (hyperparameters)

In [26]:
# Try different numbers of estimators (trees)... (no cross-validation)
np.random.seed(42)
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    model = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accuracy on test set: {model.score(X_test, y_test) * 100:.2f}%") # ":.2f" redondeo a 2 decimales
    print("")

Trying model with 10 estimators...
Model accuracy on test set: 78.95%

Trying model with 20 estimators...
Model accuracy on test set: 81.58%

Trying model with 30 estimators...
Model accuracy on test set: 76.32%

Trying model with 40 estimators...
Model accuracy on test set: 77.63%

Trying model with 50 estimators...
Model accuracy on test set: 86.84%

Trying model with 60 estimators...
Model accuracy on test set: 78.95%

Trying model with 70 estimators...
Model accuracy on test set: 78.95%

Trying model with 80 estimators...
Model accuracy on test set: 78.95%

Trying model with 90 estimators...
Model accuracy on test set: 82.89%



In [27]:
# 5. Using Random.seed inside the RandomForestClassifier
# Try different amount of n_estimators (one of our hyperparameters)
higher = -1
best_n_estimators = 0
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    clf = RandomForestClassifier(n_estimators = i, random_state = np.random.seed(1234)).fit(X_train, y_train)
    print(f"Model accuracy on test set: {clf.score(X_test, y_test)*100:.2f}%")
    if clf.score(X_test, y_test) > higher:
        higher = clf.score(X_test, y_test)
        best_n_estimators = i
    print("")
print("Best tested number of estimators is:", best_n_estimators)

Trying model with 10 estimators...
Model accuracy on test set: 78.95%

Trying model with 20 estimators...
Model accuracy on test set: 80.26%

Trying model with 30 estimators...
Model accuracy on test set: 80.26%

Trying model with 40 estimators...
Model accuracy on test set: 77.63%

Trying model with 50 estimators...
Model accuracy on test set: 78.95%

Trying model with 60 estimators...
Model accuracy on test set: 78.95%

Trying model with 70 estimators...
Model accuracy on test set: 78.95%

Trying model with 80 estimators...
Model accuracy on test set: 78.95%

Trying model with 90 estimators...
Model accuracy on test set: 77.63%

Best tested number of estimators is: 20


In [28]:
# Exporting the model
import pickle

# Save an existing model to file
pickle.dump(model, open("random_forest_model_1.pkl", "wb")) # wb: write binary

In [29]:
# Load a saved model and make a prediction
loaded_model = pickle.load(open("random_forest_model_1.pkl", "rb")) # rb: read binary
loaded_model.score(X_test, y_test)

0.8289473684210527